In [6]:
import requests
import time
import json
import selenium.webdriver.support.ui as ui
from selenium.webdriver.common.keys import Keys
import threading
import queue
import random
import datetime

from urllib.error import URLError, HTTPError
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from pprint import pprint
from selenium.webdriver.common.alert import Alert
from bs4 import BeautifulSoup
#from config.site_config import *

In [7]:
DRIVER_PATH = './config/chromedriver'
CHROME_OPTIONS = Options()

#driver.implicitly_wait(3)

In [8]:
driver = webdriver.Chrome(executable_path=DRIVER_PATH)
#driver.maximize_window()
#driver.get('https://www.saramin.co.kr/')

/var/folders/37/07y8_bp17kj20xnw5dtdsmgm0000gn/T/ipykernel_77884/652209598.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=DRIVER_PATH)


In [ ]:
# searchType=search
# searchword=컴퓨터비전            # 검색어
# loc_mcd=101000,102000         # 지역, 101000(서울),102000(경기)
# company_cd=0,1,2,3,4,5,6,7    # 채용제공구분 / 9(파견/대행), 10(헤드헌팅)
# search_optional_item=y
# search_done=y
# panel_count=y
# recruitPage=1                 # 현재 페이지뷰
# recruitSort=relation          # 정렬방법
# recruitPageCount=100          # 표시 개수
# inner_com_type=
# show_applied=                 # 지원한 공고 표시
# quick_apply=                  # 빠른 입사지원
# except_read=                  # 조회한 페이지 제외
# ai_head_hunting=
# mainSearch=n

In [9]:
search_word = "컴퓨터비전"
recruit_page = 1
recruit_page_count = 100
company_cd='0,1,2,3,4,5,6,7'

In [10]:
search_link = 'https://www.saramin.co.kr/zf_user/search/recruit\
?search_area=main\
&search_done=y\
&search_optional_item=n\
&searchType=search\
&recruitSort=relation\
&searchword={}\
&recruitPage={}\
&recruitPageCount={}\
&company_cd={}\
&mainSearch=y'.format(search_word, recruit_page, recruit_page_count, company_cd)

In [11]:
search_link

'https://www.saramin.co.kr/zf_user/search/recruit?search_area=main&search_done=y&search_optional_item=n&searchType=search&recruitSort=relation&searchword=컴퓨터비전&recruitPage=1&recruitPageCount=100&company_cd=0,1,2,3,4,5,6,7&mainSearch=y'

In [12]:
driver.get(search_link)

In [13]:
html = driver.page_source

In [14]:
soup = BeautifulSoup(html, 'html.parser')

In [89]:
# nth-chile (1~100)
title = soup.select('div.content > div:nth-child(1) > div.area_job > h2 > a')
company = soup.select('div.content > div:nth-child(1) > div.area_corp > strong > a')

In [90]:
title[0]['title']

'[주35H/재택/탄력] 컴퓨터비전, Embedded 개발자 (신입,병특)'

In [91]:
company[0]

<a class="track_event data_layer" data-data_layer="recruit_com|com_name" data-track_event="total_search|search_recruit|com_info_btn" href="/zf_user/company-info/view?csn=aGRYbWRuMHlxV21kMTVNVjNCYWkxZz09" target="_blank">
                            (주)에프에스솔루션                        </a>

In [103]:
test_string = company[0].string
test_string

'\n                            (주)에프에스솔루션                        '

In [105]:
test_string = test_string.replace(' ', '')
test_string

'\n(주)에프에스솔루션'

In [106]:
test_string = test_string.replace('\n', '')
test_string

'(주)에프에스솔루션'